Starting with basics variables using sagemaker sdk

In [1]:
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

setting the prefix to where I will upload my ready model

In [2]:
prefix = "biodata/sample"

Installing Transformers to get sentiment analysis ready model

In [3]:
!pip install Transformers

Getting sentiment analysis model

In [4]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [5]:
sentiment_analysis.save_pretrained("./model")

Compressing the model to the project root directory

In [6]:
!cd model && tar czvf ../model.tar.gz *

config.json
pytorch_model.bin
special_tokens_map.json
tokenizer.json
tokenizer_config.json
vocab.txt


Uploading the compressed model to S3

In [7]:
s3_model_path = sess.upload_data(
    path="model.tar.gz", bucket=bucket, key_prefix=prefix
)

In [8]:
print(s3_model_path)

s3://sagemaker-eu-west-1-910416587115/biodata/sample/model.tar.gz


Creating the model object, which I will use to deploy the model to an endpoint

>Please review the inference script in `inference-code/inference.py` this is how to use ower ready endpoint containers for your models, there is an option to build your own inference container and use any inference script you want

In [9]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    model_data=s3_model_path,
    role=role,
    source_dir="inference-code",
    entry_point="inference.py",
    py_version="py3",
    framework_version="1.7.1"
)

Deploying the model to SageMaker as SageMaker endpoint

In [10]:
predictor = model.deploy(initial_instance_count=1, instance_type="ml.m5.large")

------!

`model.deploy` returns a predictor object that contains all the needed information to use for running inference requests.

In [11]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

Inference using the predictor

In [12]:
result = predictor.predict([
    "testing this 1",
    "testing this 2"
])

In [13]:
print(result)

[[{'label': 'NEGATIVE', 'score': 0.02231668308377266}, {'label': 'POSITIVE', 'score': 0.9776833057403564}], [{'label': 'NEGATIVE', 'score': 0.026645060628652573}, {'label': 'POSITIVE', 'score': 0.9733549356460571}]]


Getting the endpoint name from the predictor object, which is available in sagemaker console as well

In [14]:
print(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


pytorch-inference-2022-11-03-11-53-38-665


Run inference using boto3 without having the preidctor object

In [15]:
!pip install boto3

In [16]:
import json
import boto3

runtime_client = boto3.client('runtime.sagemaker')

In [17]:
payload = json.dumps(["testing this first", "this is a second inference request"])
print(payload)

["testing this first", "this is a second inference request"]


In [18]:
response = runtime_client.invoke_endpoint(EndpointName=predictor.endpoint,
                                                  ContentType='application/json',
                                                  Body=payload)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [19]:
result = json.loads(response['Body'].read().decode())
print(result)

[[{'label': 'NEGATIVE', 'score': 0.024029599502682686}, {'label': 'POSITIVE', 'score': 0.9759703874588013}], [{'label': 'NEGATIVE', 'score': 0.9758997559547424}, {'label': 'POSITIVE', 'score': 0.02410019002854824}]]


Optional cleanup as the endpoint costs

In [ ]:
#predictor.delete_endpoint()